# Statistical comparison of errors

In [1]:
import os
import warnings

import geopandas as gpd
import matplotlib.pyplot as plt
import momepy as mm
import numpy as np
import pandas as pd
import scipy.stats

from core import stats, utils

Create a folder for evaluation data.

In [2]:
os.makedirs("../../evaluation/", exist_ok=True)

In [16]:
# which variables to evaluate
eval_vars = [
    "edge_count",
    "edge_length",
    "node_count",
    "avg_degree",
    "stroke_count",
    "stroke_length_sum",
    "stroke_length_max",
    "coordinate_count",
]

# which methods to evaluate
methods_to_evaluate = [
    "revised_manual",
    "cityseer",
    "original",
    "osmnx",
    "parenx-voronoi",
    "parenx-skeletonize",
    "neatnet",
]

methods_to_compare = [
    "cityseer",
    "osmnx",
    "parenx-voronoi",
    "parenx-skeletonize",
    "neatnet",
]

Generate evaluation grids and fill the with evaluation variables. At the same time, compute Euclidean distance between each method and a manual baseline.

In [4]:
for fua in utils.fua_city:
    print(f"Generating for {fua}")
    # read in base data
    meta = utils.read_sample_data()
    geom = meta.loc[meta.eFUA_ID == fua, "geometry"]
    city = meta.loc[meta.eFUA_ID == fua, "eFUA_name"].values[0]

    gdf_orig = utils.read_original(fua)
    proj_crs = gdf_orig.crs

    # Make grid
    base_grid = utils.make_grid(fua, 9, proj_crs)

    # get info on cells with revised data
    deltas = gpd.read_file(f"../../revision/{fua}/deltas_updated.gpkg")

    # read results from all methods into dict
    methods = {}

    for method in methods_to_evaluate:
        print(f"   Reading in results for {method}")
        gdf = utils.read_results(fua, method, proj_crs)

        # print("     getting graph")
        gdf = gdf[~gdf.normalize().duplicated()].copy().reset_index(drop=True)
        G = mm.gdf_to_nx(gdf, length="length", integer_labels=True)

        nodes, edges = mm.nx_to_gdf(G)

        # add node degrees
        # print("     adding node degree")
        nodes = stats.add_node_degree(nodes, G)

        # add stroke IDs
        # print("     measuring coins")
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            if method == "parenx-skeletonize":
                coins = mm.COINS(
                    edges.set_geometry(edges.simplify(0.5)),
                    angle_threshold=120,
                    flow_mode=True,
                )
            else:
                coins = mm.COINS(edges, angle_threshold=120, flow_mode=True)
        edges["stroke_id"] = coins.stroke_attribute()
        stroke_gdf = coins.stroke_gdf()

        methods[method] = {}
        methods[method]["gdf"] = gdf
        methods[method]["graph"] = G
        methods[method]["nodes"] = nodes
        methods[method]["edges"] = edges

        ### grid with stats eval for this method only
        grid = base_grid.copy()

        # print("     measuring grid edge")
        # add ratio columns to grid
        grid[["edge_count", "edge_length", "coordinate_count"]] = grid.apply(
            lambda x: stats.get_edge_stats(edges, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # print("     measuring grid node")
        grid[["node_count", "node_degrees", "avg_degree"]] = grid.apply(
            lambda x: stats.get_node_stats(nodes, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # print("     measuring stroke")
        grid[["stroke_count", "stroke_length_sum", "stroke_length_max"]] = grid.apply(
            lambda x: stats.get_stroke_stats(edges, stroke_gdf, x.geometry),  # noqa: B023
            axis=1,
            result_type="expand",
        )

        # add info on which cells have to be verified still
        grid["to_verify_total"] = deltas["to_verify_total"]

        # save grid to dict
        methods[method]["grid"] = grid

        # save to a file
        grid.to_file(f"../../evaluation/{fua}.gpkg", layer=method)

Generating for 1133
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results for osmnx
   Reading in results for parenx-voronoi
   Reading in results for parenx-skeletonize
   Reading in results for neatnet
Generating for 869
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results for osmnx
   Reading in results for parenx-voronoi
   Reading in results for parenx-skeletonize
   Reading in results for neatnet
Generating for 4617
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results for osmnx
   Reading in results for parenx-voronoi
   Reading in results for parenx-skeletonize
   Reading in results for neatnet
Generating for 809
   Reading in results for revised_manual
   Reading in results for cityseer
   Reading in results for original
   Reading in results 

## Statistical comparison

Compare the results with the manually simplified networks using xi correlation.

In [18]:
for fua in utils.fua_city:
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(f"../../evaluation/{fua}.gpkg", layer=method)

    # plot the distance by eval variable
    fig, axs = plt.subplots(8, figsize=(6, 18))

    for i, eval_var in enumerate(eval_vars):
        stat_result = pd.DataFrame(columns=["statistics", "pvalue"])
        for method in methods_to_compare:
            result = scipy.stats.chatterjeexi(
                data[method][eval_var].fillna(0),
                data["revised_manual"][eval_var].fillna(0),
            )
            stat_result.loc[method] = [result.statistic, result.pvalue]

        if (stat_result.pvalue > 0.01).any():
            print(fua, eval_var, "pvalue issue")
        stat_result.statistics.plot.barh(ax=axs.flat[i])
        axs.flat[i].axvline(stat_result.statistics.max(), color="coral", linestyle="--")
        axs.flat[i].set_xlabel(f"xi for {eval_var}")
        plt.tight_layout()
        fig.savefig(
            f"../../plots/evaluation/{fua}/xi_correlation.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.close()

Getting tables.

In [19]:
tables = {}
for fua in utils.fua_city:
    tables[fua] = {}
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(f"../../evaluation/{fua}.gpkg", layer=method)

    for eval_var in eval_vars:
        stat_result = pd.DataFrame(columns=["statistics", "pvalue"])
        for method in methods_to_compare + ["original"]:
            result = scipy.stats.chatterjeexi(
                data[method][eval_var].fillna(0),
                data["revised_manual"][eval_var].fillna(0),
            )
            stat_result.loc[method] = [result.statistic, result.pvalue]

        tables[fua][eval_var] = stat_result

Combinidng the tables.

In [20]:
combined = pd.DataFrame(columns=["variable", "case", "original"] + methods_to_compare)
combined[combined.columns.drop("variable")] = combined[
    combined.columns.drop("variable")
].astype(float)

i = 0
for fua in utils.fua_city:
    for eval_var in eval_vars:
        combined.loc[i] = tables[fua][eval_var].statistics
        combined.loc[i, ["variable", "case"]] = [eval_var, fua]
        i += 1

/var/folders/2f/fhks6w_d0k556plcv3rfmshw0000gn/T/ipykernel_29613/2817628644.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'edge_count' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined.loc[i, ["variable", "case"]] = [eval_var, fua]


Preparing the final output

In [21]:
combined["case"] = combined["case"].map(utils.fua_city)

In [22]:
combined = combined.set_index(["variable", "case"]).sort_index(level=0)
combined

original  cityseer     osmnx  \
variable          case                                           
avg_degree        Aleppo          0.680606  0.818574  0.728464   
                  Auckland        0.447330  0.705007  0.503962   
                  Bucaramanga     0.677106  0.763449  0.700261   
                  Douala          0.577302  0.822224  0.670818   
                  Liège           0.626935  0.780371  0.660202   
                  Salt Lake City  0.395056  0.582392  0.440178   
                  Wuhan           0.636976  0.708078  0.658766   
coordinate_count  Aleppo          0.833924  0.880297  0.874250   
                  Auckland        0.726488  0.799259  0.748882   
                  Bucaramanga     0.826543  0.842533  0.831420   
                  Douala          0.775258  0.898635  0.866832   
                  Liège           0.816854  0.842961  0.823710   
                  Salt Lake City  0.643042  0.706844  0.706059   
                  Wuhan           0.712192  0.731988  0.740962   
edge_count        Aleppo          0.628744  0.867955  0.825293   
                  Auckland        0.390791  0.719233  0.580544   
                  Bucaramanga     0.434369  0.839773  0.608383   
                  Douala          0.478361  0.897037  0.795965   
                  Liège           0.580529  0.801724  0.696586   
                  Salt Lake City  0.378158  0.657040  0.609734   
                  Wuhan           0.503475  0.724863  0.686363   
edge_length       Aleppo          0.875160  0.872793  0.873709   
                  Auckland        0.864817  0.865031  0.861626   
                  Bucaramanga     0.895524  0.862933  0.878969   
                  Douala          0.925766  0.935593  0.910692   
                  Liège           0.828329  0.826616  0.819554   
                  Salt Lake City  0.631732  0.640824  0.613787   
                  Wuhan           0.751018  0.723589  0.742146   
node_count        Aleppo          0.466668  0.877052  0.747914   
                  Auckland        0.246818  0.652091  0.423286   
                  Bucaramanga     0.307062  0.829077  0.456878   
                  Douala          0.378188  0.857659  0.678693   
                  Liège           0.452059  0.801063  0.565712   
                  Salt Lake City  0.264283  0.635013  0.503133   
                  Wuhan           0.378689  0.745883  0.598808   
stroke_count      Aleppo          0.865414  0.848545  0.820405   
                  Auckland        0.721319  0.729348  0.646925   
                  Bucaramanga     0.882188  0.824323  0.825944   
                  Douala          0.925851  0.853266  0.779272   
                  Liège           0.806491  0.782788  0.755740   
                  Salt Lake City  0.633071  0.621884  0.553637   
                  Wuhan           0.746355  0.705715  0.681939   
stroke_length_max Aleppo          0.873171  0.841503  0.733601   
                  Auckland        0.816872  0.767580  0.610769   
                  Bucaramanga     0.905139  0.836224  0.677540   
                  Douala          0.931164  0.823506  0.657246   
                  Liège           0.844628  0.814699  0.684001   
                  Salt Lake City  0.778443  0.762973  0.605920   
                  Wuhan           0.921815  0.790377  0.650646   
stroke_length_sum Aleppo          0.841629  0.799322  0.682700   
                  Auckland        0.726223  0.662355  0.541792   
                  Bucaramanga     0.887058  0.796892  0.659252   
                  Douala          0.873784  0.750064  0.587731   
                  Liège           0.785878  0.737821  0.638145   
                  Salt Lake City  0.567158  0.583962  0.413487   
                  Wuhan           0.832268  0.653147  0.532346   

                                  parenx-voronoi  parenx-skeletonize   neatnet  
variable          case                                                          
avg_degree        Aleppo                0.8

Doing the same for Euclidean distance between distributions

In [23]:
all_deltas = {}
for fua in utils.fua_city:
    all_deltas[fua] = {}
    data = {}
    for method in methods_to_evaluate:
        data[method] = gpd.read_file(f"../../evaluation/{fua}.gpkg", layer=method)

    # get euclidean distance between the distributions
    for eval_var in eval_vars:
        all_deltas[fua][eval_var] = {}
        for method in ["original"] + methods_to_compare:
            delta_comp = (
                data["revised_manual"][eval_var] - data[method][eval_var]
            ) ** 2

            delta = np.sqrt((delta_comp).sum())
            all_deltas[fua][eval_var][method] = delta

Plot the figures.

In [26]:
for fua in utils.fua_city:
    for eval_var in eval_vars:
        # plot the distance by eval variable
        fig, axs = plt.subplots(8, figsize=(6, 18))
        for i, eval_var in enumerate(eval_vars):
            s = pd.Series(all_deltas[fua][eval_var])
            s.plot.barh(ax=axs.flat[i])
            axs.flat[i].axvline(s.min(), color="coral", linestyle="--")
            axs.flat[i].set_xlabel(eval_var)
        axs[-1].set_xscale("log")
        plt.tight_layout()
        fig.savefig(
            f"../../plots/evaluation/{fua}/euclidean_distance.png",
            dpi=300,
            bbox_inches="tight",
        )
        plt.close()

Combine all together to get a single table.

In [27]:
combined_deltas = pd.DataFrame(
    columns=["variable", "case", "original"] + methods_to_compare
)
combined_deltas[combined_deltas.columns.drop("variable")] = combined_deltas[
    combined_deltas.columns.drop("variable")
].astype(float)

i = 0
for fua in utils.fua_city:
    for eval_var in eval_vars:
        combined_deltas.loc[i] = [eval_var, fua] + list(
            all_deltas[fua][eval_var].values()
        )
        i += 1

In [28]:
combined_deltas["case"] = combined_deltas["case"].map(utils.fua_city)

In [29]:
combined_deltas = combined_deltas.set_index(["variable", "case"]).sort_index(level=0)

In [30]:
combined_deltas.round(2)

original    cityseer       osmnx  \
variable          case                                                
avg_degree        Aleppo              85.59       31.67       58.65   
                  Auckland            33.70       17.34       25.11   
                  Bucaramanga         65.62       27.13       59.54   
                  Douala              38.65       14.97       26.02   
                  Liège               83.43       35.50       62.22   
                  Salt Lake City      43.51       30.64       35.25   
                  Wuhan              127.35       76.40      104.16   
coordinate_count  Aleppo            1382.17      610.82      733.47   
                  Auckland          2454.29      577.00      777.37   
                  Bucaramanga       2164.92      590.31      701.81   
                  Douala            1614.90      403.34      637.64   
                  Liège             2273.62      704.08      788.49   
                  Salt Lake City    1647.17      598.69      755.63   
                  Wuhan             1603.38      971.13      975.79   
edge_count        Aleppo            1061.56      222.52      240.78   
                  Auckland          1891.28      128.01      264.50   
                  Bucaramanga       1900.47      130.84      317.79   
                  Douala            1414.40      106.20      243.87   
                  Liège             1718.62      160.82      241.07   
                  Salt Lake City    1252.82      160.90      245.53   
                  Wuhan             1200.25      377.47      397.69   
edge_length       Aleppo           16519.57    15090.91    17315.99   
                  Auckland          6512.24     5125.48     6619.60   
                  Bucaramanga       7616.64     7111.25     8090.81   
                  Douala            5597.17     5225.22     6096.87   
                  Liège            12283.41    11084.78    12342.97   
                  Salt Lake City   16890.06    16414.89    17126.52   
                  Wuhan            23539.23    22053.92    23830.15   
node_count        Aleppo             976.61      143.90      173.04   
                  Auckland          1816.19       71.11      214.34   
                  Bucaramanga       1851.55      107.05      286.16   
                  Douala            1384.78       77.34      222.10   
                  Liège             1651.35       93.15      197.84   
                  Salt Lake City    1171.78       76.76      181.95   
                  Wuhan             1164.40      207.73      233.84   
stroke_count      Aleppo             104.24      118.55      282.17   
                  Auckland            83.91       55.15      145.21   
                  Bucaramanga         59.97       68.91      166.80   
                  Douala              46.78       59.82      165.64   
                  Liège               99.29       81.60      138.88   
                  Salt Lake City      85.60       80.30      175.99   
                  Wuhan              117.53      120.58      227.99   
stroke_length_max Aleppo          274171.86   531032.67   471476.90   
                  Auckland        103500.48   210457.83   148941.63   
                  Bucaramanga     136894.67   237001.63   235645.00   
                  Douala          224463.20   268294.23   300266.03   
                  Liège           232284.04   628972.24   259137.82   
                  Salt Lake City  240447.83   245564.65   290182.83   
                  Wuhan           609814.50   973655.01  1047796.28   
stroke_length_sum Aleppo          400970.27   532750.58   514895.18   
                  Auckland        177731.19   429900.30   241472.85   
                  Bucaramanga     155142.15   229189.78   249912.94   
                  Douala          233675.13   290668.54   333883.65   
                  Liège           811448.09  1042451.16   824507.47   
                  Salt Lake City  331715.47   339930.14   3